In [41]:
#Qiskit imports
from qiskit import QuantumCircuit, transpile
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Pauli, pauli_basis, SuperOp, PTM, Operator
#from qiskit.providers.aer.noise import NoiseModel, pauli_error
#from qiskit.providers.fake_provider import FakeVigoV2
#from qiskit.providers.aer import Aer, AerSimulator
from random import random

#python imports
from random import choice, choices
from itertools import product, permutations, cycle
from scipy.optimize import curve_fit, nnls
from matplotlib import pyplot as plt
import numpy as np

plt.style.use("ggplot")

In [42]:
def get_noise_model():
    from random import random, choices
    from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError, pauli_error, depolarizing_error, thermal_relaxation_error)
    from qiskit.quantum_info import Pauli, pauli_basis
    #from qiskit.providers.aer.noise import NoiseModel, pauli_error
    twoqubit_errorops = [Pauli('XI')]
    twoqubit_errorprobs = [0.05]
    #create normalized error model
    twoqubit_error_template = [(op, p) for op,p in zip(twoqubit_errorops, twoqubit_errorprobs)]+[(Pauli("II"), 1-sum(twoqubit_errorprobs))]
    twoqubit_error = pauli_error(twoqubit_error_template)
    noise_model = NoiseModel()
    #add error model to two-qubit gates.
    noise_model.add_all_qubit_quantum_error(twoqubit_error, ['cx','cz'])
    return (noise_model, twoqubit_error_template, [])

In [43]:
def add_CNOT(prog, num):
    for _ in range(num):
        prog.cx(0,1)
    return prog

['H 0', 'H 1', 'S 1', 'Z 0', 'CNOT 0 1', 'Z 0', 'S_DAGGER 0', 'H 0', 'X 0', 'MEASURE 0 ro[0]', 'MEASURE 1 ro[1]']
{(0, 1): 470, (1, 0): 507, (0, 0): 24, (1, 1): 23}
['H 0', 'H 1', 'S 1', 'Y 0', 'CNOT 0 1', 'Y 0', 'CNOT 0 1', 'X 1', 'S_DAGGER 1', 'H 1', 'H 0', 'X 0', 'X 1', 'MEASURE 0 ro[0]', 'MEASURE 1 ro[1]']
{'11': 925, '01': 99}

prog = QuantumCircuit(2)
prog.h(0)
prog.h(1)
prog.s(1)
prog.y(0)
prog.cx(0,1)
prog.y(0)
prog.cx(0,1)
prog.x(1)

prog2 = QuantumCircuit(2)
prog2.s(1)
prog2 = prog2.inverse()
prog = prog.compose(prog2)
prog.h(1)
prog.h(0)
prog.x(0)
prog.x(1)
prog.measure_all

prog.measure_all()
backend = GenericBackendV2(num_qubits=5, coupling_map=[(3, 4), (4, 3), (3, 1), (1, 3), (1, 2), (2, 1), (0, 1), (1, 0)])
backend = AerSimulator()
(noise_model, twoqubit_error_template, singlequbit_error_template) = get_noise_model()
#noise_model = NoiseModel()
counts = backend.run([prog], shots=1024, noise_model = noise_model).result().get_counts()
print(counts)
print(prog)

{'11': 923, '01': 101}
        ┌───┐┌───┐     ┌───┐     ┌───┐ ┌───┐            ░ ┌─┐   
   q_0: ┤ H ├┤ Y ├──■──┤ Y ├──■──┤ H ├─┤ X ├────────────░─┤M├───
        ├───┤├───┤┌─┴─┐└───┘┌─┴─┐├───┤┌┴───┴┐┌───┐┌───┐ ░ └╥┘┌─┐
   q_1: ┤ H ├┤ S ├┤ X ├─────┤ X ├┤ X ├┤ Sdg ├┤ H ├┤ X ├─░──╫─┤M├
        └───┘└───┘└───┘     └───┘└───┘└─────┘└───┘└───┘ ░  ║ └╥┘
meas: 2/═══════════════════════════════════════════════════╩══╩═
                                                           0  1 


In [44]:
len(["IXZ", "IYI", "XII", "IYY", "ZII", "YII", "IZZ", "IYX", "ZYI", "IZI", "YZI", "ZXI", "IYZ", "ZZI", "IIY", "IZX", "IXX", "YXI", "IIZ", "IZY", "IXY", "YYI", "XXI", "XYI", "IXI", "IIX", "XZI"])
["ZI", "XZ", "YY", "ZX", "XX", "IY", "YI", "IX", "YX", "XY", "YZ", "IZ", "XI", "ZY", "ZZ"]

['ZI',
 'XZ',
 'YY',
 'ZX',
 'XX',
 'IY',
 'YI',
 'IX',
 'YX',
 'XY',
 'YZ',
 'IZ',
 'XI',
 'ZY',
 'ZZ']

: 

In [1]:
a = [1,2,3]
b = [4,5,6,7,8]
ab = [[i+j for i in a] for j in b]
print(ab)

[[5, 6, 7], [6, 7, 8], [7, 8, 9], [8, 9, 10], [9, 10, 11]]
